In [ ]:
import numpy as np
import torch
from ax.plot.render import plot_config_to_html
from ax.service.ax_client import AxClient, ObjectiveProperties
from ax.utils.measurement.synthetic_functions import hartmann6
from ax.utils.notebook.plotting import init_notebook_plotting, render

In [ ]:
ax_client = AxClient()

In [ ]:
# Function that we're trying to optimise
def evaluation_function(parameters):
    x = np.array([parameters.get(f"x{i}") for i in range(6)])
    return {
        "hartmann6": hartmann6(x),
        "l2norm": np.sqrt((x**2).sum()),
    }


In [ ]:
# Objectives
objectives = {
    "hartmann6": ObjectiveProperties(minimize=True),
}

In [ ]:
# Parameters
parameters = []
for i in range(6):
    parameters.append(
        {
            "name": f"x{i}",
            "type": "range",
            "bounds": [0.0, 1.0],
        }
    )

In [ ]:
# Constraints
parameter_constraints = [
    "x1 + x2 <= 2.0",
]
outcome_constraints = [
    "l2norm <= 1.25",
]

In [ ]:
# Define experiment
ax_client.create_experiment(
    name="hartmann_test_experiment",
    parameters=parameters,
    objectives=objectives,
    parameter_constraints=parameter_constraints,
    outcome_constraints=outcome_constraints,
)

In [ ]:
# Run experiment
for i in range(25):
    parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(
        trial_index=trial_index,
        raw_data=evaluation_function(parameters),
    )

In [ ]:
init_notebook_plotting()
render(
    ax_client.get_optimization_trace(
        objective_optimum=hartmann6.fmin,
    ),
)